# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [268]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

aag  aarefajohari  aayega  abandoned  abandons  abattoirs  abbott  \
april       0             0       1          3         0          0       0   
february    0             0       0          0         0          0       0   
january     0             0       0          0         0          0       0   
july        0             0       0          0         0          1       0   
june        1             0       0          0         0          0       1   
march       0             0       0          0         1          0       0   
may         0             1       0          0         0          0       0   

          abbotts  abc  abcs  ...  來自  武汉肺炎  고양이  고양이스타그램  궈낙아쉥일축하훼  깍두기  제가  \
april           0    1     0  ...   0     0    0        0         0    0   0   
february        0    0     1  ...   1     1    0        0         0    1   1   
january         0    2     1  ...   0     0    0        0         0    0   0   
july            0    5     0  ...   0     0    0        0         1    0   0   
june            0    1     0  ...   0     0    0        0         0    0   0   
march           1    0     0  ...   0     0    1        1         0    0   0   
may             0    2     0  ...   0     0    0        0         0    0   0   

          페르시아  𝗔𝘂𝘁𝗵𝗼𝗿𝘀  𝗧𝗶𝘁𝗹𝗲  
april        0        0      0  
february     1        0      0  
january      0        0      0  
july         0        0      0  
june         0        1      1  
march        0        0      0  
may          0        0      0  

[7 rows x 9935 columns]

In [269]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

In [270]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

april  february  january  july  june  march  may
aag               0         0        0     0     1      0    0
aarefajohari      0         0        0     0     0      0    1
aayega            1         0        0     0     0      0    0
abandoned         3         0        0     0     0      0    0
abandons          0         0        0     0     0      1    0

In [271]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [272]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [273]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=80)
lda.print_topics()

[(0,
  '0.005*"china" + 0.004*"deaths" + 0.003*"health" + 0.003*"wuhan" + 0.003*"spread" + 0.003*"amp" + 0.003*"world" + 0.003*"outbreak" + 0.003*"time" + 0.003*"death"'),
 (1,
  '0.006*"deaths" + 0.003*"fauci" + 0.003*"mask" + 0.003*"texas" + 0.003*"masks" + 0.003*"president" + 0.003*"going" + 0.003*"world" + 0.003*"china" + 0.003*"did"'),
 (2,
  '0.005*"hoax" + 0.005*"china" + 0.003*"outbreak" + 0.003*"flu" + 0.003*"health" + 0.003*"said" + 0.002*"pence" + 0.002*"update" + 0.002*"dont" + 0.002*"confirmed"'),
 (3,
  '0.000*"china" + 0.000*"health" + 0.000*"deaths" + 0.000*"going" + 0.000*"did" + 0.000*"world" + 0.000*"outbreak" + 0.000*"confirmed" + 0.000*"think" + 0.000*"spread"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [274]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' 
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [275]:
data_nouns_adj = pd.DataFrame(data_clean.text.apply(nouns_adj))
data_nouns_adj

text
april     calls health sector hand profit crisis superch...
february  i outbreak things s mexico canada hotel occupa...
january   cancels league dreamcatcher i liberals wall tr...
july      fyi surpasses york deaths day weekend start up...
june      sport care access projects people lives devast...
march     christians demons lives vulnerable attention m...
may       air risk home yourice risk cause cops hundreds...

In [281]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said', 'ppl', 'video', 'tweet']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

aarefajohari  abattoirs  abbott  abbotts  abc  abcs  abe  abeg  \
april                0          0       0        0    1     0    2     0   
february             0          0       0        0    0     1    0     1   
january              0          0       0        0    2     1    0     0   
july                 0          1       0        0    4     0    0     0   
june                 0          0       1        0    0     0    0     0   
march                0          0       0        1    0     0    0     0   
may                  1          0       0        0    1     0    0     0   

          abets  ability  ...  來自  武汉肺炎  고양이  고양이스타그램  궈낙아쉥일축하훼  깍두기  제가  \
april         0        1  ...   0     0    0        0         0    0   0   
february      0        1  ...   1     1    0        0         0    1   1   
january       1        0  ...   0     0    0        0         0    0   0   
july          0        0  ...   0     0    0        0         1    0   0   
june          0        1  ...   0     0    0        0         0    0   0   
march         0        0  ...   0     0    1        1         0    0   0   
may           0        0  ...   0     0    0        0         0    0   0   

          페르시아  𝗔𝘂𝘁𝗵𝗼𝗿𝘀  𝗧𝗶𝘁𝗹𝗲  
april        0        0      0  
february     1        0      0  
january      0        0      0  
july         0        0      0  
june         0        1      1  
march        0        0      0  
may          0        0      0  

[7 rows x 5718 columns]

In [293]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [294]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"hedge" + 0.004*"pence" + 0.003*"bioweapon" + 0.003*"fear" + 0.003*"insertions" + 0.003*"novel" + 0.003*"flights" + 0.002*"cause" + 0.002*"humans" + 0.002*"asians"'),
 (1,
  '0.005*"rioters" + 0.005*"protests" + 0.005*"dems" + 0.004*"lockdowns" + 0.004*"bigcity" + 0.004*"riots" + 0.004*"violence" + 0.004*"police" + 0.003*"cause" + 0.003*"recovery"'),
 (2,
  '0.007*"lab" + 0.004*"agency" + 0.003*"unemployment" + 0.002*"job" + 0.002*"uk" + 0.002*"claims" + 0.002*"fauci" + 0.002*"facts" + 0.002*"intelligence" + 0.002*"april"'),
 (3,
  '0.006*"fauci" + 0.004*"dr" + 0.004*"texas" + 0.002*"job" + 0.002*"infections" + 0.002*"surge" + 0.002*"joe" + 0.002*"hurricane" + 0.002*"hospitals" + 0.002*"florida"')]

* Topic 0: hurricane Hanna
* Topic 1: claims, fear and hedge
* Topic 2: danger
* Topic 3: protests

In [295]:
# Let's take a look at which topics each text contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmn.index))

[(2, 'april'),
 (0, 'february'),
 (0, 'january'),
 (3, 'july'),
 (3, 'june'),
 (3, 'march'),
 (1, 'may')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0:  hurricane Hanna [July]
* Topic 1: claims, fear and hedge [January, February, March,April]
* Topic 2: danger [June]
* Topic 3: protests [May]